<h1 style="text-align:center;"><b>This notebook formats output.json to be a bit more readable</b></h1>
<h3><i>Notes:</i><br>Ukraine is included because they have been evaluated in the past, could be in the future, and are listed on the website like any evaluated country. It would not make sense to change the code to exclude Ukraine specifically or null values more generally, because those countries are listed on the website for a reason.</h3><h4>The climateactiontracker.org website states:<br>"We have suspended our rating for Ukraine as the country defends itself from the unlawful Russian invasion. We stand in solidarity with the people of Ukraine at this terrible time."</h4>
<h3>Gabon is no longer tracked, but is worth including for the sake of consistency (with the Ukraine decision).</h3>

In [21]:
import pandas as pd

# Load the JSON data into a DataFrame
df = pd.read_json("output.json")

# Normalize the "indicators" column
indicators_df = pd.json_normalize(df['indicators'])

# Further normalize each indicator
normalized_indicators = []
for i, indicator in enumerate(indicators_df.columns):
    temp_df = pd.json_normalize(indicators_df[indicator])
    temp_df.columns = [f'indicator_{i + 1}_{col}' for col in temp_df.columns]
    normalized_indicators.append(temp_df)

# Concatenate all normalized indicators
indicators_df = pd.concat(normalized_indicators, axis=1)

# Merge "term" and "term_details" columns
for i in range(1, len(normalized_indicators) + 1):
    term_col = f'indicator_{i}_term'
    term_details_col = f"indicator_{i}_term_details"
    
    if term_col in indicators_df.columns and term_details_col in indicators_df.columns:
        # Merge term and term_details, handling NaN
        indicators_df[term_col] = indicators_df[term_col].fillna("") + " " + indicators_df[term_details_col].fillna("")
        indicators_df[term_col] = indicators_df[term_col].str.strip()  # Remove any extra spaces
        indicators_df.drop(columns = [term_details_col,], inplace = True)  # Remove term_details column

# Concatenate the original DataFrame with the fully normalized indicators DataFrame
df = pd.concat([df.drop(columns = ['indicators']), indicators_df.drop(columns = "indicator_4_metric")], axis = 1)

# Display the DataFrame
df.style


,country_name,overall_rating,indicator_1_term,indicator_1_value,indicator_1_metric,indicator_2_term,indicator_2_value,indicator_2_metric,indicator_3_term,indicator_3_value,indicator_3_metric,indicator_4_term,indicator_4_value
0,Argentina,Critically insufficient,Policies and action against modelled domestic pathways,Highly insufficient,< 4°C World,NDC target against modelled domestic pathways,Highly insufficient,< 4°C World,NDC target against fair share,Critically insufficient,4°C+ World,Climate finance,Not assessed
1,Australia,Insufficient,Policies and action against modelled domestic pathways,Insufficient,< 3°C World,NDC target against modelled domestic pathways,Insufficient,< 3°C World,NDC target against fair share,Insufficient,< 3°C World,Climate finance,Critically insufficient
2,Bhutan,Almost Sufficient,Policies and action against fair share,1.5°C compatible,< 1.5°C World,CAT estimate against modelled domestic pathways,1.5°C global least cost,< 1.5°C World,CAT estimate against fair share,Almost Sufficient,< 2°C World,Climate finance,Not applicable
3,Brazil,Insufficient,Policies and action against fair share,Insufficient,< 3°C World,NDC target against modelled domestic pathways,Almost Sufficient,< 2°C World,NDC target against fair share,Almost Sufficient,< 2°C World,Climate finance,Not applicable
4,Canada,Insufficient,Policies and action against modelled domestic pathways,Insufficient,< 3°C World,NDC target against modelled domestic pathways,Almost Sufficient,< 2°C World,NDC target against fair share,Insufficient,< 3°C World,Climate finance,Highly insufficient
5,Chile,Almost Sufficient,Policies and action against modelled domestic pathways,Almost Sufficient,< 2°C World,Conditional NDC target against modelled domestic pathways,Almost Sufficient,< 2°C World,Unconditional NDC target against fair share,Almost Sufficient,< 2°C World,Climate finance,Not assessed
6,China,Highly insufficient,Policies and action against fair share,Insufficient,< 3°C World,NDC target against modelled domestic pathways,Highly insufficient,< 4°C World,NDC target against fair share,Insufficient,< 3°C World,Climate finance,Not assessed
7,Colombia,Insufficient,Policies and action against fair share,Insufficient,< 3°C World,NDC target against modelled domestic pathways,Insufficient,< 3°C World,NDC target against fair share,Almost Sufficient,< 2°C World,Climate finance,Not applicable
8,Costa Rica,Almost Sufficient,Policies and action against fair share,1.5°C compatible,< 1.5°C World,NDC target against modelled domestic pathways,Almost Sufficient,< 2°C World,NDC target against fair share,1.5°C compatible,< 1.5°C World,Climate finance,Not applicable
9,EU,Insufficient,Policies and action against modelled domestic pathways,Insufficient,< 3°C World,NDC target against modelled domestic pathways,Insufficient,< 3°C World,NDC target against fair share,Insufficient,< 3°C World,Climate finance,Insufficient
